In [13]:
import sqlite3
import csv
import os
import re
from pathlib import Path
from datetime import datetime

def main():
    """Création de la base de données cycliste avec toutes les tables et importation automatique des données CSV."""
    # Créer ou se connecter à la base de données
    db_path = Path("cyclist_database.db")
    
    # Supprimer la base de données si elle existe déjà
    if db_path.exists():
        db_path.unlink()
    
    # Établir la connexion
    conn = sqlite3.connect(db_path)
    cursor = conn.cursor()
    
    # Activer les contraintes de clés étrangères
    cursor.execute("PRAGMA foreign_keys = ON;")
    
    # Création de la table User
    cursor.executescript("""
    CREATE TABLE User (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        first_name TEXT NOT NULL,
        last_name TEXT NOT NULL,
        email TEXT UNIQUE NOT NULL,
        password TEXT NOT NULL,
        is_staff BOOLEAN DEFAULT 0
    );
    """)
    
    # Création de la table CyclistInfo
    cursor.executescript("""
    CREATE TABLE CyclistInfo (
        user_id INTEGER PRIMARY KEY,
        gender TEXT CHECK(gender IN ('male', 'female')),
        age INTEGER CHECK(age > 0),
        weight REAL CHECK(weight > 0),
        height REAL CHECK(height > 0),
        FOREIGN KEY (user_id) REFERENCES User(id) ON DELETE CASCADE
    );
    """)
    
    # Création de la table Test
    cursor.executescript("""
    CREATE TABLE Test (
        test_id INTEGER PRIMARY KEY AUTOINCREMENT,
        user_id INTEGER NOT NULL,
        test_type TEXT CHECK(test_type IN ('protocol_1', 'protocol_2', 'incremental', 'wingate')),
        test_date DATETIME DEFAULT CURRENT_TIMESTAMP,
        FOREIGN KEY (user_id) REFERENCES User(id) ON DELETE CASCADE
    );
    """)
    
    # Création de la table Performances
    cursor.executescript("""
    CREATE TABLE Performances (
        performance_id INTEGER PRIMARY KEY AUTOINCREMENT,
        test_id INTEGER NOT NULL,
        user_id INTEGER NOT NULL,
        time INTEGER NOT NULL,
        oxygen REAL NOT NULL,
        power REAL NOT NULL,
        cadence REAL NOT NULL,
        heart_rate REAL NOT NULL,
        respiration_freq REAL NOT NULL,
        vo2_max REAL,
        FOREIGN KEY (test_id) REFERENCES Test(test_id) ON DELETE CASCADE,
        FOREIGN KEY (user_id) REFERENCES User(id) ON DELETE CASCADE
    );
    """)
    
    # Insérer les utilisateurs
    insert_users(cursor)
    
    # Charger et insérer les données du dossier data
    data_folder = Path("data")  # Utilisez le nom correct de votre dossier de données
    if data_folder.exists() and data_folder.is_dir():
        import_csv_data(cursor, conn, data_folder)
    else:
        print(f"Le dossier {data_folder} n'existe pas ou n'est pas un dossier. Tentative d'importation depuis le dossier courant.")
        # Si le dossier data n'existe pas, essayez d'importer depuis le dossier courant
        import_csv_data(cursor, conn, Path("."))
    
    # Valider les changements et fermer la connexion
    conn.commit()
    conn.close()
    
    print("Base de données créée avec succès!")

def insert_users(cursor):
    """Insère les utilisateurs et les informations des cyclistes."""
    # Insérer 7 utilisateurs (un pour chaque sujet dans les fichiers CSV)
    users_data = []
    cyclist_info = []
    
    for i in range(1, 8):
        user = (
            f"Subject{i}", 
            f"Cyclist", 
            f"subject{i}@example.com", 
            f"password{i}", 
            0
        )
        users_data.append(user)
        
        # Alternance entre homme et femme
        gender = "male" if i % 2 == 1 else "female"
        # Âge entre 25 et 35 ans
        age = 25 + i
        # Poids en kg
        weight = 70.0 + (i * 1.5) if gender == "male" else 58.0 + (i * 1.2)
        # Taille en cm
        height = 175.0 + (i * 0.8) if gender == "male" else 165.0 + (i * 0.6)
        
        cyclist_data = (i, gender, age, weight, height)
        cyclist_info.append(cyclist_data)
    
    # Ajouter un compte admin
    users_data.append(("Admin", "System", "admin@cyclistapp.com", "admin2024!", 1))
    
    # Insérer les utilisateurs
    cursor.executemany(
        "INSERT INTO User (first_name, last_name, email, password, is_staff) VALUES (?, ?, ?, ?, ?)",
        users_data
    )
    
    # Insérer les informations des cyclistes
    cursor.executemany(
        "INSERT INTO CyclistInfo (user_id, gender, age, weight, height) VALUES (?, ?, ?, ?, ?)",
        cyclist_info
    )

def import_csv_data(cursor, conn, data_folder):
    """Importe automatiquement toutes les données des fichiers CSV du dossier dans la base de données."""
    # Chercher tous les fichiers CSV dans le dossier
    all_csv_files = [f for f in os.listdir(data_folder) if f.endswith('.csv')]
    print(f"Fichiers CSV trouvés: {all_csv_files}")
    
    # Patterns pour extraire les informations des noms de fichiers
    sbj_pattern = re.compile(r'sbj_(\d+)_([^.]+)\.csv')
    bonus_pattern = re.compile(r'bonus_sbj_([^.]+)\.csv')
    
    # Dictionnaire pour suivre les tests déjà créés
    test_ids = {}
    
    # Parcourir tous les fichiers CSV
    for csv_file in all_csv_files:
        file_path = data_folder / csv_file
        
        # Extraire les informations du nom de fichier
        sbj_match = sbj_pattern.match(csv_file)
        bonus_match = bonus_pattern.match(csv_file)
        
        if sbj_match:
            # Fichier standard sujet
            subject_id = int(sbj_match.group(1))
            test_type_code = sbj_match.group(2).lower()
            
            # Convertir le code du test en type de test valide
            if test_type_code.lower() == 'i':
                test_type = 'protocol_1'
            elif test_type_code.lower() == 'ii':
                test_type = 'protocol_2'
            elif 'incremental' in test_type_code.lower():
                test_type = 'incremental'
            elif 'wingate' in test_type_code.lower():
                test_type = 'wingate'
            else:
                print(f"Type de test non reconnu dans {csv_file}: {test_type_code}, utilisation de 'protocol_1' par défaut")
                test_type = 'protocol_1'
            
            # Créer une clé pour ce test spécifique
            test_key = f"sbj_{subject_id}_{test_type}"
            
        elif bonus_match:
            # Fichier bonus
            test_code = bonus_match.group(1).lower()
            
            if 'incremental' in test_code:
                subject_id = 8  # Attribuer un nouvel ID pour le sujet bonus
                test_type = 'incremental'
            elif '4x' in test_code:
                subject_id = 9  # Autre sujet bonus
                test_type = 'protocol_1'
            elif '8x' in test_code:
                subject_id = 10  # Autre sujet bonus
                test_type = 'protocol_2'
            else:
                subject_id = 11  # ID par défaut pour autres bonus
                test_type = 'protocol_1'
                print(f"Type de test bonus non reconnu: {test_code}, utilisation de 'protocol_1' par défaut")
            
            # Créer une clé pour ce test spécifique
            test_key = f"bonus_{test_code}"
            
        else:
            print(f"Format de nom de fichier non reconnu: {csv_file}. Importation ignorée.")
            continue
        
        # Vérifier si ce test existe déjà, sinon le créer
        if test_key not in test_ids:
            # S'assurer que l'utilisateur existe pour les sujets bonus
            if subject_id > 7:
                # Vérifier si l'utilisateur existe déjà
                cursor.execute("SELECT id FROM User WHERE id = ?", (subject_id,))
                if not cursor.fetchone():
                    # Créer un nouvel utilisateur pour le sujet bonus
                    cursor.execute(
                        "INSERT INTO User (id, first_name, last_name, email, password, is_staff) VALUES (?, ?, ?, ?, ?, ?)",
                        (subject_id, f"Subject{subject_id}", "Bonus", f"subject{subject_id}@example.com", f"password{subject_id}", 0)
                    )
                    
                    # Créer également une entrée dans CyclistInfo
                    gender = "male" if subject_id % 2 == 1 else "female"
                    cursor.execute(
                        "INSERT INTO CyclistInfo (user_id, gender, age, weight, height) VALUES (?, ?, ?, ?, ?)",
                        (subject_id, gender, 30, 75.0, 175.0)
                    )
            
            # Créer un nouveau test
            cursor.execute(
                "INSERT INTO Test (user_id, test_type) VALUES (?, ?)",
                (subject_id, test_type)
            )
            test_id = cursor.lastrowid
            test_ids[test_key] = test_id
            print(f"Nouveau test créé: ID={test_id}, Type={test_type} pour Sujet={subject_id}")
        else:
            test_id = test_ids[test_key]
        
        # Importer les données du fichier CSV
        try:
            with open(file_path, 'r', encoding='utf-8') as f:
                csv_reader = csv.DictReader(f)
                
                # Vérification des colonnes requises
                required_cols = ['time', 'Power', 'Oxygen', 'Cadence', 'HR', 'RF']
                if not csv_reader.fieldnames:
                    print(f"Attention: Le fichier {csv_file} est vide ou mal formaté.")
                    continue
                
                missing_cols = [col for col in required_cols if col not in csv_reader.fieldnames]
                
                if missing_cols:
                    print(f"Attention: Colonnes manquantes dans {csv_file}: {missing_cols}")
                    continue
                
                # Préparation de la requête d'insertion
                insert_query = """
                INSERT INTO Performances 
                (test_id, user_id, time, power, oxygen, cadence, heart_rate, respiration_freq)
                VALUES (?, ?, ?, ?, ?, ?, ?, ?)
                """
                
                # Insertion des données
                performance_data = []
                for row in csv_reader:
                    try:
                        performance_data.append((
                            test_id,
                            subject_id,
                            int(row['time']),
                            float(row['Power']),
                            float(row['Oxygen']),
                            float(row['Cadence']),
                            float(row['HR']),
                            float(row['RF'])
                        ))
                        
                        # Exécuter par lots pour de meilleures performances
                        if len(performance_data) >= 1000:
                            cursor.executemany(insert_query, performance_data)
                            performance_data = []
                            conn.commit()
                            
                    except (ValueError, KeyError) as e:
                        print(f"Erreur lors du traitement d'une ligne dans {csv_file}: {e}")
                
                # Insérer les données restantes
                if performance_data:
                    cursor.executemany(insert_query, performance_data)
                
                print(f"Données de {csv_file} importées avec succès.")
                
        except Exception as e:
            print(f"Erreur lors de l'importation de {csv_file}: {e}")

if __name__ == "__main__":
    main()

Fichiers CSV trouvés: ['sbj_1_Wingate.csv', 'bonus_sbj_4x.csv', 'sbj_4_incremental.csv', 'bonus_sbj_incremental.csv', 'sbj_2_Wingate.csv', 'sbj_3_I.csv', 'sbj_2_incremental.csv', 'sbj_1_incremental.csv', 'sbj_6_Wingate.csv', 'sbj_5_incremental.csv', 'sbj_3_Wingate.csv', 'sbj_3_incremental.csv', 'sbj_5_II.csv', 'sbj_2_II.csv', 'sbj_3_II.csv', 'sbj_5_I.csv', 'sbj_4_Wingate.csv', 'sbj_7_Wingate.csv', 'sbj_4_I.csv', 'sbj_6_I.csv', 'sbj_1_I.csv', 'sbj_6_incremental.csv', 'sbj_7_II.csv', 'sbj_4_II.csv', 'sbj_2_I.csv', 'sbj_5_Wingate.csv', 'sbj_1_II.csv', 'bonus_sbj_8x.csv', 'sbj_7_incremental.csv', 'sbj_7_I.csv', 'sbj_6_II.csv']
Nouveau test créé: ID=1, Type=wingate pour Sujet=1
Données de sbj_1_Wingate.csv importées avec succès.
Nouveau test créé: ID=2, Type=protocol_1 pour Sujet=9
Données de bonus_sbj_4x.csv importées avec succès.
Nouveau test créé: ID=3, Type=incremental pour Sujet=4
Données de sbj_4_incremental.csv importées avec succès.
Nouveau test créé: ID=4, Type=incremental pour Suj